In [ ]:
#!pip3 install pickle5 utm
import pickle5 as pickle
import matplotlib.pyplot as plt
import numpy as np
import utm

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
#!ls drive/'My Drive'
drive_path = 'drive/My Drive/assignment/'
data_0_path = drive_path + "data_0.pickle"
data_1_path = drive_path + "data_1.pickle"

with open(data_0_path, "rb") as fh:
  data = pickle.load(fh)

with open(data_1_path, "rb") as fh:
  data1 = pickle.load(fh)

In [ ]:
data

In [ ]:
x = data["GM.acc.xyz.x"][0]
y = data["GM.acc.xyz.y"][0]
z = data["GM.acc.xyz.z"][0]

plt.plot(x,"r")
plt.plot(y,"g")
plt.plot(z,"b")

In [ ]:
lats = np.concatenate(data["GM.lat_int"].values).ravel()
lons = np.concatenate(data["GM.lon_int"].values).ravel()
vels = np.concatenate(data["GM.obd.spd_veh.value"].values).ravel()
lats1 = np.concatenate(data1["GM.lat_int"].values).ravel()
lons1 = np.concatenate(data1["GM.lon_int"].values).ravel()
vels1 = np.concatenate(data1["GM.obd.spd_veh.value"].values).ravel()

filter_length = 5
vel_smooth = np.convolve(np.array(vels),np.ones(filter_length)/filter_length,"same")

vels_avg=list()
for v in data["GM.obd.spd_veh.value"].values:
  vels_avg.append(v.mean())

In [ ]:
plt.figure(figsize=(12,12))
plt.plot(vel_smooth,"g")

In [ ]:
plt.figure(figsize=(15,15))
m, b = np.polyfit(np.array(vels_avg,dtype=np.float),data["IRI_mean"].to_numpy(dtype=np.float), 1)
plt.plot(np.array(vels_avg,dtype=np.float), m*np.array(vels_avg,dtype=np.float) + b,"r")
plt.scatter(vels_avg,data["IRI_mean"].to_numpy(),alpha=0.5)
plt.xlabel("average velocity")
plt.ylabel("IRI mean")

In [ ]:
from scipy import optimize

method_2 = "leastsq"

def calc_R(xc, yc):
    """ calculate the distance of each 2D points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def f_2(c):
    """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(*c)
    return Ri - Ri.mean()

def fit_circle(x,y):
  x_m = np.mean(x)
  y_m = np.mean(y)

  center_estimate = x_m, y_m
  center_2, ier = optimize.leastsq(f_2, center_estimate)

  xc_2, yc_2 = center_2
  Ri_2       = calc_R(*center_2)
  R_2        = Ri_2.mean()
  residu_2   = sum((Ri_2 - R_2)**2)
  return R_2, center_2

def get_dists(utm_east, utm_north):
  c=0
  dists = np.zeros(utm_east.shape)
  inds = np.zeros(utm_east.shape)

  for i in range(utm_east.shape[0]-1):
    d = np.linalg.norm((utm_north[i]-utm_north[i+1], utm_east[i]-utm_east[i+1]))
    if d>1:
      inds[i]=1
    if d==0:
      inds[i]=2
    dists[i] = d
  print("nm of points with dist greater than 1m: {}, 0m: {}".format(inds[np.where(inds==1)[0]].sum(),inds[np.where(inds==2)[0]].sum()/2))
  return dists, inds


def find_min_support_index(utm_east, utm_north,min_support_distance):
  dists, inds = get_dists(utm_east,utm_north)
  # remove coincident coordinates
  dists = dists[np.where(dists!=2)]
  support_indexes = [[],[]]
  support_lengths = []
  curr_dist,start_ind = 0,0

  for i in range(dists.shape[0]):
    if curr_dist>min_support_distance or inds[i]==1:
      support_indexes[0].append(start_ind)
      support_indexes[1].append(i)
      support_lengths.append(curr_dist)
      start_ind = i+1
      curr_dist = 0
      
    curr_dist += dists[i]


  return np.array(support_indexes), np.array(support_lengths)

def fit_circles_to_track(lats,lons, min_support_distance=20):
  stride = 1000
  r,c = [],[]
  utm_east, utm_north = to_utm(lats,lons)
  for i in range(0,utm_north.shape[0],stride):
    idx = find_min_support_index(utm_east, utm_north,20, i)
    n,e = utm_north[i:idx],utm_east[i:idx]
    rt,ct = fit_circle(n,e)
    r.append(rt)
    c.append(ct)
  return np.array(r),np.array(c)

def to_utm(lats,lons):
  [utm_east, utm_north, utm_zone, utm_letter] = utm.from_latlon(lats,lons)
  utm_east -= utm_east.min()
  utm_north -= utm_north.min()
  return utm_east, utm_north

In [ ]:
utm_east, utm_north = to_utm(lats,lons)

In [ ]:
#dists, inds = get_dists(utm_east[::1000],utm_north[::1000])
#plt.hist(dists[np.where(inds!=2)[0]],bins=100,range=(0,10))



In [ ]:
support_indexes, support_lengths =  find_min_support_index(utm_east[::100], utm_north[::100],20)


In [ ]:
plt.hist(support_lengths,bins=100, range=(0,30))

In [ ]:
utm_east, utm_north = to_utm(lats,lons)

fig, ax = plt.subplots()

for i in range(r_c.shape[1]):
  if r_c[1][i][0] < 8000 :
    if r_c[1][i][1] < 8000: 
      #print(r_c[1][i][0],r_c[1][i][1])
      ax.scatter(r_c[1][i][0], r_c[1][i][1],c="r") 

#ax.scatter(utm_north[::100],utm_east[::100])


In [ ]:
r_c[1].shape

In [ ]:
BBox = (np.min(lons),np.max(lons),      
         np.min(lats), np.max(lats))

track = plt.imread(drive_path + "map.png")


def get_mean(d):
  v_mean = list()
  for v in d.values:
    v_mean.append(v.mean())
  v_mean=np.array(v_mean)

  v_mean_scaled=(v_mean-v_mean.mean())/v_mean.std()
  v_mean_scaled-=v_mean_scaled.min()
  return np.array(v_mean_scaled)**3

v_mean = get_mean(data["GM.obd.spd_veh.value"])
v_mean1 = get_mean(data1["GM.obd.spd_veh.value"])


fig, ax = plt.subplots(figsize=(16, 12))
for i in range(data["GM.lat_int"].values.shape[0]):
  ax.scatter(data["GM.lon_int"].values[i][::700], data["GM.lat_int"].values[i][::700],
             zorder=1, alpha= 0.1, c='b', s=v_mean[i])
  
for i in range(data1["GM.lat_int"].values.shape[0]):
  ax.scatter(data1["GM.lon_int"].values[i][::700], data1["GM.lat_int"].values[i][::700],
             zorder=1, alpha= 0.1, c='g', s=v_mean1[i])

ax.set_title('Plot of latitute/longitude data (1&2) on Copenhagen map')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
#ax.legend("Blue -> Data 0 \nGreen -> Data 1")
ax.imshow(track, zorder=0, extent = BBox, aspect= 'auto')